# Exercise session notes

SVD is a genralization of eigenvalue decomposition.

Given

$$ A = U \cdot D \cdot V^{T} $$

it holds that

$$ \left\{ \begin{array}{l}
    A \cdot V_i = d_i \cdot u_i\\
    A^{T} \cdot U_i = d_i \cdot v_i
\end{array}\right. 
\iff 
\left\{ \begin{array}{l}
    A \cdot A^{T} \cdot U_i = d_{i}^{2} u_{i} \\
    A^{T} \cdot A \cdot V_i = d_{i}^{2} v_{i} 
\end{array}\right. .$$

Exercise: prove that $r(\tilde{A}) = 2$, where $\tilde{A}_2 = U_2 \cdot D_2 \cdot V_2^T$.

# Problem 1 (SVD Theory)

In [46]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import copy
import random

In [4]:
A = np.array([
        [8, 7,      1, np.NaN, 4],
        [9, 7,      2, 5,      6],
        [1, 4,      9, np.NaN, 3],
        [3, np.NaN, 8, 5,      4],
        [np.NaN, 3, np.NaN, 9, 9],
        [5, 1, 4, 10, np.NaN]
    ])

print(A.shape)
data

(6, 5)


NameError: name 'data' is not defined

$A \in \mathbb{R}^{M \times N}$.

1. $ K = AA^{T}, K \in \mathbb{R}^{M \times M} $ indicates row-row correlations (movie-movie similarities). Every element (i, j) is just the dot product between movies i and j. Could consider using cosine similarity instead of the dot product in order not to get distracted by the absolute rating when computing similarities (movie pairs with higher ratings lead to larger dot products than equally similar movies but with lower ratings).
2. $ L = A^{T}A, L \in \mathbb{R}^{N \times N}$ indicates column-column correlations (user-user similarities).
3. $ A = UDV^{T}, U: (M \times M), D: (M \times N), V^{T}: (N \times N) $; in our case: $ U: (6 \times 6), D: (6 \times 5), V^{T}: (5 \times 5) $
4. Performing SVD on the rating column would help us isolate some latent concepts. It will, for instance, yield a matrix mapping movies to their belonging to a particular type (or genre), as well as one mapping users to their affinities towards certain genres. Main goal: low-rank approximation of original matrix. Compute missing ratings to use for recommending movies.
5. D contains the singular values $\sigma_{i}$ on its diagonal. It's possible to keep all nonzero singular values, but it's generally more useful to just keep the K largest ones. In our case, we could consider keeping the first three or two.
6. The matrix U can be seen as a description of every movie as a composition of some latent variables.
7. The matrix V can be seen as a description of every user as a composition of some latent variables (e.g. affinities to certain movie genres).

In [ ]:
A_imputed = np.nan_to_num(A)
A_imputed[A_imputed == 0] = 5.5
A_imputed

In [ ]:
def full_svd(matrix):
    U, d, V = np.linalg.svd(matrix, full_matrices=True)
    D = np.zeros(matrix.shape)
    D[:d.shape[0],:d.shape[0]] = np.diag(d)
    return U, D, V

In [ ]:
U, D, V = full_svd(A_imputed)

In [ ]:
%precision 2
U, D, V.T

In [ ]:
reconstruction = np.dot(U, np.dot(D, V))
reconstruction

In [ ]:
np.allclose(A_imputed, reconstruction)

In [ ]:
def scatter_label(data, labels):
    """Helper function for pretty scatterplots."""
    plt.scatter(data[:, 0], data[:, 1], s=60)
    for label, x, y in zip(labels, data[:, 0], data[:, 1]):
        plt.annotate(
            label, 
            xy = (x, y), xytext = (20, 20),
            textcoords = 'offset points', ha = 'right', va = 'bottom',
            bbox = dict(boxstyle = 'round, pad=0.5', fc = 'cyan', alpha = 0.5),
            arrowprops = dict(arrowstyle = '->', connectionstyle = 'arc3,rad=0'))

In [ ]:
U_2 = U[:,:2]
movie_labels = ['American Pie', 'Shrek', 'Titanic', 'The godfather', 'Avatar', 'Star Wars']
scatter_label(U_2, movie_labels)
    
_ = plt.title("8. Representations of movies using first two singular values.", 
              size=20, y=1.1)

We can interpret the above plot's X axis as a (sort-of) degree of realism, while the Y axis can be seen as a distinction between drama (positive) and comedy (negative).

In [ ]:
# Idea: analyze massive data and detect cultists who enjoy cult
# classics which some critics consider really bad (e.g. The Room, The Big Lebowski, etc.).

# Idea: analyze stuff user hasn't yet seen. Avoid stuff with very high or very low ratings.

V_2 = V[:,:2]
user_labels = ['Ben', 'Tom', 'John', 'Fred', 'Jack']
scatter_label(V_2, user_labels)

John and Jack prefer more serious movies, while Fred, Tom, and Ben are into comedies. Out of them, Fred seems to be the most well-rounded, but with still quite a strong inclination towards sci-fi.

The second singular value represents the comedy/drama division, while the first one seems to represent the degree of realism.

In [ ]:
A_approx = np.dot(U[:,:3], np.dot(D_full[:3,:], V))
print(A_imputed)
print(A_approx)
error = np.linalg.norm(A_approx - A_imputed)
print("Approximation error (frobenius): %f" % error)

## Bob wants to join the system!

In [ ]:
Bob = np.array([[1, np.NaN, np.NaN, 6, np.NaN, 10]])
Bob_imputed = np.nan_to_num(Bob)
Bob_imputed[Bob_imputed == 0] = 5.5
Bob_imputed

In [ ]:
# TODO(andrei) Do this in a prettier way.
A_bob_imputed = np.hstack((A_imputed, Bob_imputed.T))
A_bob_imputed

In [ ]:
U, D, V = full_svd(A_bob_imputed)
scatter_label(U[:,:2], movie_labels)

In [ ]:
user_labels.append('Bob')
scatter_label(V[:,:2], user_labels)

Bob seems to be close to Fred, in that he also likes more serous movies, but he's even farther than the rest since he absolutely hates comedies such as American Pie.

In [ ]:
A_bob_approx = np.dot(U[:,:3], np.dot(D[:3,:], V))
print("Approximate rating matrix including Bob:\n\n", A_bob_approx, "\n")
error_frob = np.linalg.norm(A_approx - A_imputed, ord='fro')
error_euclid = np.linalg.norm(A_approx - A_imputed, ord=2)

print(("Approximation errors after adding Bob to system:\n" +
      "\t- Frobenius norm: %f\n" +
      "\t- Euclidean norm: %f\n") % (error_frob, error_euclid))

We use the highest predicted rating which was missing in Bob's original ratings in order to recommend him a movie.

In [ ]:
# Get Bob's rating column!
bob_index = user_labels.index('Bob')
Bob_new_ratings = A_bob_approx[:, bob_index]

# Keep track of the original row indices in the column...
Bob_candidates = np.array([np.arange(Bob.shape[1]), Bob_new_ratings])
# ...and only keep rows for movies Bob didn't see yet.
Bob_candidates = Bob_candidates[:, np.isnan(Bob[0])]

# Pick the row with the highest rating.
best_entry = Bob_candidates[:, np.argmax(Bob_candidates[1,:])]
# And then grab the movie index.
best_index = int(best_entry[0])

print("Bob should watch %s!" % movie_labels[best_index])

In closing, Bob's ratings do affect our prediction system, since they add more information (and also a little bit more confusion, due to a slightly larger approximation error) to our data matrix.

# Problem 2: Collaborative Filtering

In [34]:
def load_ratings(data_file):
    ratings = []
    with open(data_file, 'r') as file:
        header = file.readline()
        print("Header: %s" % header)
        for line in file:
            key, value_string = line.split(",")
            rating = int(value_string)
            row_string, col_string = key.split("_")
            row = int(row_string[1:])
            col = int(col_string[1:])
            
            ratings.append((row - 1, col - 1, rating))
            
    print("Finished loading ratings.")
    return ratings

def ratings_to_matrix(ratings, matrix_rows, matrix_cols):
    print("Building [%d x %d] rating matrix." % (matrix_rows, matrix_cols))
    matrix = np.zeros([matrix_rows, matrix_cols])
    for (row, col, rating) in ratings:
        matrix[row, col] = rating
        
    print("Finished building rating matrix.")
    return matrix

In [42]:
# Rows are items
ITEM_COUNT = 1000

# Columns are users
USER_COUNT = 10000

VALIDATION_RATIO = 0.1
TRAINING_RATIO = 1 - VALIDATION_RATIO

RANDOM_SEED = 0xC0FFEE

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

In [43]:
all_ratings = load_ratings('data/cf/data_train.csv')

Header: Id,Prediction

Finished loading ratings.


In [44]:
data_matrix = ratings_to_matrix(all_ratings, USER_COUNT, ITEM_COUNT)

Building [10000 x 1000] rating matrix.
Finished building rating matrix.


In [45]:
all_ratings_count = len(all_ratings)
print("We have [%d] data points." % all_ratings_count)

validation_count = int(VALIDATION_RATIO * all_ratings_count)
training_count = all_ratings_count - validation_count
print("Will use [%d] data points for validation, and [%d] for training." %
     (validation_count, training_count))

We have [1388107] data points.
Will use [138810] data points for validation, and [1249297] for training.


In [47]:
def shuffle_split_data(all_data, training_count, validation_count):
    """Returns a split consisting of (training, validation) data, created
    after shuffling the input data.
    """
    to_shuffle = copy.copy(all_data)
    random.shuffle(to_shuffle)
    return to_shuffle[:training_count], to_shuffle[training_count:]

In [49]:
train, validation = shuffle_split_data(all_ratings, training_count, validation_count)
train_matrix = ratings_to_matrix(train, USER_COUNT, ITEM_COUNT)
validation_matrix = ratings_to_matrix(validation, USER_COUNT, ITEM_COUNT)

assert len(train_matrix[train_matrix != 0]) == training_count
assert len(validation_matrix[validation_matrix != 0]) == validation_count

Building [10000 x 1000] rating matrix.
Finished building rating matrix.
Building [10000 x 1000] rating matrix.
Finished building rating matrix.


In [24]:
actual_ratings = r[r != 0]
len(actual_ratings)

67